### c) Descripción de los operadores que se pueden aplicar para la función expandir(nodos).

__Mover disco de una torre a otra:__

Descripción:

1) Seleccionar una torre de origen (de las tres torres disponibles: A, B, C).<br>$S_0$
2) Seleccionar una torre de destino (distinta de la torre de origen y que cumpla con las reglas del juego, es decir, el disco a mover debe ser más pequeño que el disco en la cima de la torre de destino).<br>$S_m$
3) Verificar que las torres de origen no esté vacía y que la torre de destino acepte el disco a mover según las reglas del juego.<br>$S_0 > 2$
4) Expandir: $s: \rightarrow \{ s_i1, ... , s_in \}$
5) Prueba de éxito: $meta?: s \rightarrow  verdad|falso$
6) Generar una nueva configuración del juego después de aplicar este movimiento.<br>$M: (A, B, C) \rightarrow (A', B', C')$
7) Agregar la nueva configuración a la lista de nodos expandidos.<br>$L = L \cup \{N\}$

### f) Proponer y describir una heurística.

__Distancia de Hamming__: contar los discos que se encuentran en diferentes posiciones entre la configuración inicial y la final e incrementar en uno el coste por cada disco que se encuentre en una posición diferente entre ambas configuraciones.

Descripción:

1) Calcular la distancia de Hamming entre la configuración actual y la configuración final del problema.
2) Cuantos más discos estén en diferentes posiciones, mayor será la distancia de Hamming y mayor será la estimación de la cantidad de movimientos necesarios.
3) La heurística elegirá la configuración de expansión que minimice la distancia de Hamming, es decir, que más se acerque a la configuración objetivo.

### g) Programa en Python de la búsqueda de la solución usando el algoritmo de __A* con la heurística propuesta__.

In [5]:
from queue import PriorityQueue

def calcular_distancia_hamming(configuracion_actual, configuracion_objetivo):
    distancia_hamming = 0
    for torre_actual, torre_objetivo in zip(configuracion_actual, configuracion_objetivo):
        for disco_actual, disco_objetivo in zip(torre_actual, torre_objetivo):
            if disco_actual != disco_objetivo:
                distancia_hamming += 1
    return distancia_hamming

def esFinal(configuracion, configuracion_objetivo):
    return configuracion == configuracion_objetivo

def expandir(configuracion, configuracion_objetivo):
    sucesores = []                                                                                                      # Aquí se inicializa una lista vacía llamada sucesores que se utilizará para almacenar los estados sucesores generados.
    for i in range(3):
        for j in range(3):
            if i != j and (configuracion[i] and (not configuracion[j] or configuracion[i][-1] < configuracion[j][-1])): # Esta condición asegura que solo se realicen movimientos válidos de torres según las reglas del problema.
                nuevo_estado = [list(torre) for torre in configuracion]                                                 # Aquí se crea una copia profunda del estado actual para evitar modificar el estado origina
                if nuevo_estado[i]:                                                                                     # Verificar si la lista no está vacía antes de llamar a pop()
                    nuevo_estado[j].append(nuevo_estado[i].pop())                                                       # Aquí se saca el disco de una torre y se mete en la otra
                    sucesores.append((nuevo_estado, calcular_distancia_hamming(nuevo_estado, configuracion_objetivo)))
    return sucesores

def verifica_hanoi(torres):
    for torre in torres:
        if torre != sorted(torre, reverse=True):
            return False
    return True

def ingresar_datos_hanoi():
    configuracion_inicial = []
    configuracion_objetivo = []
    for i in range(3):
        torre_inicial = input(f"Por favor, ingresa los discos en la torre inicial {i+1}, de arriba a abajo, separados por comas: ")
        torre_inicial = list(map(int, torre_inicial.split(','))) if torre_inicial else []
        configuracion_inicial.append(torre_inicial)
        
    for i in range(3):
        torre_objetivo = input(f"Por favor, ingresa los discos en la torre objetivo {i+1}, de arriba a abajo, separados por comas: ")
        torre_objetivo = list(map(int, torre_objetivo.split(','))) if torre_objetivo else []
        configuracion_objetivo.append(torre_objetivo)
    
    while not verifica_hanoi(configuracion_inicial) or not verifica_hanoi(configuracion_objetivo):
          print('No puede haber discos de menor tamaño de los que tienen encima.\nIndica los valores de esta forma: 3, 2, 1')
          configuracion_inicial, configuracion_objetivo = ingresar_datos_hanoi()
    
    return configuracion_inicial, configuracion_objetivo

def heuristicaEstatica(configuracion_inicial, configuracion_objetivo):
    frontera = PriorityQueue()                                      # Aquí se crea una cola de prioridad vacía llamada frontera. Se utilizará para almacenar los nodos que se deben explorar, ordenados por una prioridad específica.   
    frontera.put((0, configuracion_inicial))                        # Se agrega la configuración inicial a la frontera con una prioridad de 0. La prioridad 0 indica que es el nodo inicial y no tiene ningún costo hasta ahora.
    padres = {tuple(map(tuple, configuracion_inicial)): None}       # Se inicializa un diccionario llamado padres que se utilizará para mantener un registro de los padres de cada nodo. Aquí, se asigna la configuración inicial como una clave con el valor None, ya que no tiene un padre.
    costo_camino = {tuple(map(tuple, configuracion_inicial)): 0}    # Se inicializa otro diccionario llamado costo_camino que se utilizará para mantener un registro del costo acumulado de llegar a cada nodo desde la configuración inicial. Se asigna la configuración inicial con un costo de 0.
    ### contador = 0

    while not frontera.empty():                                     # El algoritmo seguirá explorando nodos hasta que no haya más nodos en la frontera para explorar.
        _, estado_actual = frontera.get()                           # Se saca el nodo con la mayor prioridad de la frontera y se asigna a estado_actual. El guion bajo _ se utiliza para descartar la prioridad, ya que no se necesita en este punto.

        if esFinal(estado_actual, configuracion_objetivo):          # Ejecuta si el estado actual es el estado objetivo. Comienza con una lista vacía llamada camino y 
            camino = []                                             # agrega el estado actual a esta lista en cada iteración del bucle while. Luego, actualiza el estado 
            while estado_actual:                                    # actual al padre del estado actual (obtenido del diccionario padres). Esto continúa hasta que 
                camino.append(estado_actual)                        # estado_actual se convierte en None, lo que significa que hemos alcanzado el estado inicial. 
                estado_actual = padres[tuple(map(tuple, estado_actual))]
            return camino[::-1]                                     # Finalmente, devuelve el camino invertido, ya que lo hemos construido desde el objetivo hacia el inicio.

        for sucesor, distancia in expandir(estado_actual, configuracion_objetivo):
            nuevo_costo = costo_camino[tuple(map(tuple, estado_actual))] + 1
            if tuple(map(tuple, sucesor)) not in costo_camino or nuevo_costo < costo_camino[tuple(map(tuple, sucesor))]:
                costo_camino[tuple(map(tuple, sucesor))] = nuevo_costo
                prioridad = nuevo_costo + distancia
                frontera.put((prioridad, sucesor))
                padres[tuple(map(tuple, sucesor))] = estado_actual

In [6]:
# Ejecutar el algoritmo A*
inicio, meta = ingresar_datos_hanoi()
solucion = heuristicaEstatica(inicio, meta)

# Imprimir la solución
for paso, estado in enumerate(solucion):
            print(f"Paso {paso + 1}: {estado}")

Paso 1: [[3, 2], [], []]
Paso 2: [[3], [2], []]
Paso 3: [[], [2], [3]]
Paso 4: [[], [], [3, 2]]


# Configuraciones inicial y objetivo
configuracion_inicial = [[3, 2, 1], [], []]
configuracion_objetivo = [[], [], [3, 2, 1]]

# Configuración 2
configuracion_inicial_2 = [[2, 1], [], [3]]
configuracion_objetivo_2 = [[], [], [3, 2, 1]]

# Configuración 3
configuracion_inicial_3 = [[3], [2], [1]]
configuracion_objetivo_3 = [[], [], [3, 2, 1]]

# Configuración 4
configuracion_inicial_4 = [[], [2, 1], [3]]
configuracion_objetivo_4 = [[], [], [3, 2, 1]]

# Configuración 5
configuracion_inicial_5 = [[5], [4], [3, 2, 1]]
configuracion_objetivo_5 = [[], [5, 4, 3], [2, 1]]